In [1]:
# Hartree-Fock program v3

     FC = eSC
     F = T + V_Ne + V_ee

In [2]:
import sys
import math
import numpy as np
from scipy import special 

In [3]:
class primitive_gaussian():
    
    def __init__(self, alpha, coeff, coordinates, l1, l2, l3):
    
        self.alpha = alpha
        self.coeff = coeff
        self. coordinates = np.array(coordinates)
        self.l1 = l1
        self.l2 = l2
        self.l3 = l3
        # Normalisation constant
        self.A = (2.0* alpha / np.pi) ** 0.75  # + other terms when we have l1, l2 ,l3 (for p orbital, d orbital)
        

In [1]:
# Fourier transform
def boys(x, n):
    if x==0:
        return 1.0/(2*n+1)
    
    else:
        return special.gammainc(n+0.5, x) * special.gamma(n+0.5) * (1.0/(2*x**(n+0.5)))

In [2]:
def electron_nuclear_attraction(molecule, atom_coordinates, Z):
    
    natoms = len(Z)
    nbasis = len(molecule)
    
    V_ne = np.zeros((nbasis, nbasis))
    
    for atom in range(natoms):
        for i in range(nbasis):            # row of matrix
            for j in range(nbasis):        # column of matrix
            
            # taking account of all the primitive(gaussian functions) bases present in the atomic slater function  
            
                nprimitives_i = len(molecule[i])   # number of gaussian describing the ith slater fucntion
                nprimitives_j = len(molecule[j])    # number of gaussian describing the jth slater function
            
                for k in range(nprimitives_i):
                    for l in range(nprimitives_j):
                    
                        c1c2 = molecule[i][k].coeff * molecule[j][l].coeff
                        N = molecule[i][k].A * molecule[j][l].A
                        p = molecule[i][k].alpha + molecule[j][l].alpha
                        q = molecule[i][k].alpha * molecule[j][l].alpha / p
                        Q = molecule[i][k].coordinates - molecule[j][l].coordinates
                        Q2 = np.dot(Q, Q)
                    
                        P = molecule[i][k].alpha * molecule[i][k].coordinates + molecule[j][l].alpha * molecule[j][l].coordinates
                        Pp = P / p
                        PG = Pp - atom_coordinates[atom]
                        PG2 = np.dot(PG, PG)
                    
                        V_ne[i,j] +=  -Z[atom] *N * c1c2* np.exp(-q*Q2) * (2.0*np.pi/p)*boys(p*PG2, 0)
                        
    return V_ne

In [6]:
# STO-3G basis for 1s orbital on hydrogen
# These are descriptions of atomic orbital basis 
                            # Alpha,           # coeff            # Coord   # angular momentum
H1_pg1a = primitive_gaussian (0.3425250914E+01, 0.1543289673E+00, [0,0,0], 0, 0, 0)
H1_pg1b = primitive_gaussian (0.6239137298E+00 , 0.5353281423E+00, [0,0,0], 0, 0, 0)
H1_pg1c = primitive_gaussian (0.1688554040E+00, 0.4446345422E+00, [0,0,0], 0, 0, 0)

H2_pg1a = primitive_gaussian (0.3425250914E+01, 0.1543289673E+00, [1.4,0,0], 0, 0, 0)
H2_pg1b = primitive_gaussian (0.6239137298E+00 , 0.5353281423E+00, [1.4,0,0], 0, 0, 0)
H2_pg1c = primitive_gaussian (0.1688554040E+00, 0.4446345422E+00, [1.4,0,0], 0, 0, 0)

H1_1s = [H1_pg1a, H1_pg1b, H1_pg1c]  # Describing the atomic orbital with slater function
H2_1s = [H2_pg1a, H2_pg1b, H2_pg1c]

Z = [1.0, 1.0]
atom_coordinates = [np.array([0, 0, 0]), np.array([1.4, 0, 0])]
molecule = [H1_1s, H2_1s]

# type(molecule)
# len(molecule[0])

print("electron_nuclear attraction matrix:\n", electron_nuclear_attraction(molecule, atom_coordinates, Z))

electron_nuclear attraction matrix:
 [[-1.88044089 -1.19483462]
 [-1.19483462 -1.88044089]]


In [7]:
# 6-31G basis for 1s and 2s orbitals on hydrogen

H1_pg1a = primitive_gaussian (0.1873113696E+02, 0.3349460434E-01, [0,0,0], 0, 0, 0)
H1_pg1b = primitive_gaussian (0.2825394365E+01  , 0.2347269535E+00, [0,0,0], 0, 0, 0)
H1_pg1c = primitive_gaussian (0.6401216923E+00, 0.8137573261E+00, [0,0,0], 0, 0, 0)
H1_pg2a = primitive_gaussian (0.1612777588E+00, 1.0000000, [0,0,0], 0, 0, 0)

H2_pg1a = primitive_gaussian (0.1873113696E+02, 0.3349460434E-01, [1.4,0,0], 0, 0, 0)
H2_pg1b = primitive_gaussian (0.2825394365E+01  , 0.2347269535E+00, [1.4,0,0], 0, 0, 0)
H2_pg1c = primitive_gaussian (0.6401216923E+00, 0.8137573261E+00, [1.4,0,0], 0, 0, 0)
H2_pg2a = primitive_gaussian (0.1612777588E+00, 1.0000000, [1.4,0,0], 0, 0, 0)

H1_1s = [H1_pg1a, H1_pg1b, H1_pg1c]
H1_2s = [H1_pg2a]
H2_1s = [H2_pg1a, H2_pg1b, H2_pg1c]
H2_2s = [H2_pg2a]

Z = [1.0, 1.0]
atom_coordinates = [np.array([0, 0, 0]), np.array([1.4, 0, 0])]
molecule = [H1_1s, H1_2s, H2_1s,  H2_2s]

print("electron_nuclear attraction matrix:\n", electron_nuclear_attraction(molecule, atom_coordinates, Z))

electron_nuclear attraction matrix:
 [[-2.35561243 -1.18063294 -1.03966339 -0.94457307]
 [-1.18063294 -1.16883952 -0.94457307 -1.03930485]
 [-1.03966339 -0.94457307 -2.35561243 -1.18063294]
 [-0.94457307 -1.03930485 -1.18063294 -1.16883952]]
